# LTH Implementation Conv2 - SVHN

In [ ]:
# parts from "Conv_2_CIFAR10_Magnitude_based_Pruning_Gaussian_Glorot_initializations", 
# https://github.com/arjun-majumdar/Lottery_Ticket_Hypothesis-TensorFlow_2

### Load packages

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

import keras
import pickle

from tensorflow.keras.layers import AveragePooling2D, Conv2D, MaxPooling2D, ReLU
from tensorflow.keras import models, layers, datasets
from tensorflow.keras.layers import Dense, Flatten, Reshape, Input, InputLayer
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.initializers import RandomNormal


%load_ext tensorboard

## Load and handle SVHN dataset

In [ ]:
# save important features of SVHN 
num_classes = 10
img_rows, img_cols = 32, 32

In [ ]:
from scipy.io import loadmat

################# Change Folder here! ##############################
# SVHN data can be found here: http://ufldl.stanford.edu/housenumbers
train = loadmat("data/numbers_train_32x32.mat")
test = loadmat("data/numbers_test_32x32.mat")

X_train = np.array(train["X"])
y_train = np.array(train["y"])

X_test = np.array(test["X"])
y_test = np.array(test["y"])

# bring into right format (shape = (nr_images, height, width, channels))
X_train = np.swapaxes(X_train, 3, 0)
X_train = np.swapaxes(X_train, 3, 1)
X_train = np.swapaxes(X_train, 3, 2)

X_test = np.swapaxes(X_test, 3, 0)
X_test = np.swapaxes(X_test, 3, 1)
X_test = np.swapaxes(X_test, 3, 2)

# change label '10' to '0'
y_train = np.array([x if x != [10] else [0] for x in y_train])
y_test = np.array([x if x != [10] else [0] for x in y_test])

# Convert datasets to floating point types-
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# Normalize the training and testing datasets-
X_train /= 255.0
X_test /= 255.0

# convert class vectors/target to binary class matrices or one-hot encoded values-
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

# split training set into train and validation (10%)
l = len(X_train)
X_val = X_train[:int(l/10)]
X_train = X_train[int(l/10):]

y_val = y_train[:int(l/10)]
y_train = y_train[int(l/10):]

# combine into real datasets
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_ds = train_ds.batch(batch_size = batch_size)

val_ds = tf.data.Dataset.from_tensor_slices((X_val, y_val))
val_ds = val_ds.batch(batch_size = batch_size)

# Functions

### Conv-2 architecture

In [ ]:
# oriented at https://keras.io/guides/customizing_what_happens_in_fit/, last retrieved 15.12.21
class CustomModel(keras.Sequential):
    
    def train_step(self, data):
        # Unpack the data.
        x, y = data
                        
        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)  # Forward pass
            # Compute the loss value
            # (the loss function is configured in `compile()`)
            loss = self.compiled_loss(y, y_pred, regularization_losses=self.losses)

        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        
        # List to hold element-wise multiplication between-
        # computed gradient and masks-
        grad_mask_mul = []
        

        # Perform element-wise multiplication between computed gradients and 0, if weight was already
        # 0 before (was pruned, very unlikely that a weight would by chance reach exactly 0!), gradient is 
        # multiplied by 1 else (in order to keep the pruned weights at 0, apply the gradient everywhere else
        for grad_layer, train_vars in zip(gradients, trainable_vars):
            grad_mask_mul.append(tf.math.multiply(grad_layer, tf.math.ceil(tf.math.abs(train_vars))))
    
        # Apply computed gradients to model's weights and biases-
        self.optimizer.apply_gradients(zip(grad_mask_mul, trainable_vars))
        
        # Update metrics (includes the metric that tracks the loss)
        self.compiled_metrics.update_state(y, y_pred)
        # Return a dict mapping metric names to current value
        return {m.name: m.result() for m in self.metrics}

In [ ]:
def conv2_cnn(initializer = tf.initializers.GlorotNormal(), custom = False):
    """
    Function to define the architecture of a neural network model
    following Conv-2 architecture for CIFAR-10 dataset and using
    provided parameter which are used to prune the model.

    Conv-2 architecture-
    64, 64, pool  -- convolutions
    256, 256, 10  -- fully connected layers

    Inputs:
    'initializer' defines which initializer to use for weights and biases
    'custom' is True if model should follow the custom fit-function and not train weights that are 0

    Output: Returns designed and compiled neural network model
    """
    if custom:
        model = CustomModel()
    else:
        model = Sequential()
    
    # first conv layer
    model.add(
        Conv2D(
            filters = 64, kernel_size = (3, 3),
            activation='relu', kernel_initializer = initializer,
            bias_initializer = initializer,
            strides = (1, 1), padding = 'same',
            input_shape=(32,32, 3)
            )
    )

    # second conv layer
    model.add(
        Conv2D(
            filters = 64, kernel_size = (3, 3),
            activation='relu', kernel_initializer = initializer,
            bias_initializer = initializer,
            strides = (1, 1), padding = 'same'
            )
    )
    
    # pooling layer
    model.add(
        MaxPooling2D(
            pool_size = (2, 2),
            strides = (2, 2)
        )
    )

    model.add(Flatten())
    
    # first dense layer
    model.add(
        Dense(
            units = 256, activation='relu',
            kernel_initializer = initializer,
            bias_initializer = initializer
        )
    )

    # second dense layer
    model.add(
        Dense(
            units = 256, activation='relu',
            kernel_initializer = initializer,
            bias_initializer = initializer
        )
    )

    # third dense layer / output layer
    model.add(
        Dense(
            units = 10, activation='softmax'
        )
    )


    # Compile CNN-
    model.compile(
        loss=tf.keras.losses.categorical_crossentropy,
        optimizer=tf.keras.optimizers.Adam(learning_rate = 0.0002),
        metrics=['accuracy']
    )


    return model

### Plot Histories

In [ ]:
def plot_histories(his1,name1,his2,name2):
    plt.plot(his1.history["accuracy"], label = "accuracy {0}".format(name1))
    plt.plot(his2.history["accuracy"], label = "accuracy {0}".format(name2))
    plt.plot(his1.history["val_accuracy"], label = "val_accuracy {0}".format(name1))
    plt.plot(his2.history["val_accuracy"], label = "val_accuracy {0}".format(name2))
    plt.legend()
    plt.title("Accuracy")
    plt.show()


    plt.plot(his1.history["loss"], label = "loss {0}".format(name1))
    plt.plot(his2.history["loss"], label = "loss {0}".format(name2))
    plt.plot(his1.history["val_loss"], label = "val_loss {0}".format(name1))
    plt.plot(his2.history["val_loss"], label = "val_loss {0}".format(name2))
    plt.legend()
    plt.title("Loss")
    plt.show()

### Prune Conv-2 model

In [ ]:
# from arjun, changed to give back model and not give back pruned weights in list

def prune_conv2(model, pruning_params_conv, pruning_params_fc, pruning_params_op):
    '''
    Function to prune top p% of trained weights using the provided parameters using
    magnitude-based weight pruning.
    
    Inputs:
    'model' is the TensorFlow 2.0 defined convolutional neural network
    'pruning_params_conv' is the percentage of weights to prune for convolutional layer
    'pruning_params_fc' is the percentage of weights to prune for dense, fully-connected layer
    'pruning_params_op' is the percentage of weights to prune for output layer

    Returns:
    pruned_model 
    '''

    ### initialize new model with 0 everywhere that's going to get changed
    pruned_model = conv2_cnn("zeros", custom = True)  

    for layer_old, layer_pruned in zip(model.layers, pruned_model.layers):   
        
        # if max_pooling or flatten
        if layer_old.get_weights() == []:
            continue
        
        # if convolutional layer
        if len(layer_old.get_weights()[0].shape) == 4:
            
            #print("conv layer: {0}, pruning rate = {1}%".format(layer_old.get_weights()[0].shape, pruning_params_conv))
            
            # shape of w is (height, width, channels, tensors), shape of b is (tensors)
            w = layer_old.get_weights()[0]
            b = layer_old.get_weights()[1] # doesn't need to be changed
            
            # Compute absolute value of 'w'
            w_abs = np.abs(w)

            # Mask values to zero which are less than 'p' in terms of magnitude
            w_abs[w_abs < np.percentile(w_abs, pruning_params_conv)] = 0

            # Where 'w_abs' equals 0, keep 0, else, replace with values of 'w'
            w_new = np.where(w_abs == 0, 0, w)
    
            layer_pruned.set_weights([w_new,b])

        # if fully-connected dense layer
        elif len(layer_old.get_weights()[0].shape) == 2 and layer_old.get_weights()[0].shape[1] != 10:

            #print("dense layer: {0}, pruning rate = {1}%".format(layer_old.get_weights()[0].shape, pruning_params_fc))
            
            # shape of w is (incoming_image_pixels, neurons), shape of b is (neurons)
            w = layer_old.get_weights()[0]
            b = layer_old.get_weights()[1] # doesn't need to be changed

            # Compute absolute value of 'w'
            w_abs = np.abs(w)

            # Mask values to zero which are less than 'p' in terms of magnitude
            w_abs[w_abs < np.percentile(w_abs, pruning_params_fc)] = 0

            # Where 'w_abs' equals 0, keep 0, else, replace with values of 'w'
            w_new = np.where(w_abs == 0, 0, w)

            layer_pruned.set_weights([w_new,b])
        
        # if output layer
        elif len(layer_old.get_weights()[0].shape) == 2 and layer_old.get_weights()[0].shape[1] == 10:

            #print("op layer: {0}, pruning rate = {1}%".format(layer_old.get_weights()[0].shape, pruning_params_op))

            # shape of w is (incoming_image_pixels, neurons), shape of b is (neurons)
            w = layer_old.get_weights()[0]
            b = layer_old.get_weights()[1] # doesn't need to be changed

            # Compute absolute value of 'w'
            w_abs = np.abs(w)

            # Mask values to zero which are less than 'p' in terms of magnitude
            w_abs[w_abs < np.percentile(w_abs, pruning_params_fc)] = 0

            # Where 'w_abs' equals 0, keep 0, else, replace with values of 'w'
            w_new = np.where(w_abs == 0, 0, w)

            layer_pruned.set_weights([w_new,b])


    return pruned_model

## Set IM-Pruning params

In [ ]:
###################### set final pruning rate here !! ###########################################
final_percentage_weights = 0.1  # at most 10% of weights should be left

# number of convolutional parameters
conv1 = 1792
conv2 = 36928

# number of fully-connected dense parameters
dense1 = 4194560
dense2 = 65792
op_layer = 2570


# total number of parameters
total_params = conv1 + conv2 + dense1 + dense2 + op_layer

final_pruned_params = final_percentage_weights * total_params 

# Lists to hold percentage of weights pruned in each round for all layers in CNN
conv_pruning = []
dense_pruning = []
op_layer_pruning = []

In [ ]:
# count necessary rounds of pruning 
n = 0

# save momentary percentage of pruned params after n rounds
loc_tot_params = total_params
loc_conv1 = conv1
loc_conv2 = conv2
loc_dense1 = dense1
loc_dense2 = dense2
loc_op_layer = op_layer

# save pruned percentages for each round until goal is reached
while loc_tot_params >= final_pruned_params:
    loc_conv1 *= 0.9    # 10% weights are pruned each round
    loc_conv2 *= 0.9    # 10% weights are pruned
    loc_dense1 *= 0.8   # 20% weights are pruned
    loc_dense2 *= 0.8   # 20% weights are pruned
    loc_op_layer *= 0.9 # 10% weights are pruned
    
    conv_pruning.append(((conv1 - loc_conv1) / conv1) * 100)
    dense_pruning.append(((dense1 - loc_dense1) / dense1) * 100)
    op_layer_pruning.append(((op_layer - loc_op_layer) / op_layer) * 100)

    loc_tot_params = loc_conv1 + loc_conv2 + loc_dense1 + loc_dense2 + loc_op_layer

    n += 1

In [ ]:
# Convert from list to np.array and round
conv_pruning = np.round(np.array(conv_pruning),3)
dense_pruning = np.round(np.array(dense_pruning),3)
op_layer_pruning = np.round(np.array(op_layer_pruning),3)

# Iterative Magnitude Pruning

### other functions

In [ ]:
# for each round of training before pruning, add history section to whole history
def add_section_to_his(his_whole, his_section):
    his_whole.history["accuracy"] += his_section.history["accuracy"]
    his_whole.history["val_accuracy"] += his_section.history["val_accuracy"]
    his_whole.history["loss"] += his_section.history["loss"]
    his_whole.history["val_loss"] += his_section.history["val_loss"]
    return his_whole

In [ ]:
# for both WT and original network, train for some epochs and save history to be able to compare 
# their performance later on
def compare_WT_to_orig(WT, orig_model, nr):
    # how many epochs they should train 
    # (about 40 to match the 6000 iterations of Frankle/Carbin, but 10 are enough)
    epochs = 10

    his_WT = WT.fit(
        train_ds,
        validation_data=val_ds,
        epochs=epochs,
        verbose = 0
        )
    
    his_orig = orig_model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=epochs,
        verbose = 0
    )
    
    # save histories
    with open('../tickets/WTs_SVHN/his_WT_s0.1_nr' + str(nr), 'wb') as file_pi:
        pickle.dump(his_WT.history, file_pi)
        
    with open('../tickets/WTs_SVHN/his_orig_s0.1_nr' + str(nr), 'wb') as file_pi:
        pickle.dump(his_orig.history, file_pi)
        
    plot_histories(his_WT, "WT", his_orig, "Orig")
    
    # use early stopping criterion from Frankle/Carbin: Minimum val_loss iteration.
    # only print results here, check later on when actually using the tickets
    print("WT has min in epoch", np.argmin(his_WT.history["val_loss"])+1, "with val_loss", np.min(his_WT.history["val_loss"]))
          
    print("Orig_model has min in epoch", np.argmin(his_orig.history["val_loss"])+1, "with val_loss", np.min(his_orig.history["val_loss"]))
          

### IMP

In [ ]:
########### set parameters here ##################
nr_winning_tickets = 20
nr_pruning_rounds = n
nr_epochs_per_round = 8

# for each WT that should be extracted:
for i in range(0, nr_winning_tickets):   
    
    print("Starting nr ", i)
    
    # initiate random network and save it
    orig_model = conv2_cnn()
    # serialize weights to HDF5
    orig_model.save_weights("../tickets/WTs_SVHN/orig_model_s0.1_nr" + str(i) + ".h5")
        
    # train several steps, prune certain percentage, reinitialize and train again until final percentage achieved
    # initiate model to be trained
    trained_model = conv2_cnn(custom = True)
    trained_model.set_weights(orig_model.get_weights()) 
        
    for j in range(nr_pruning_rounds):
        print("pruning round nr ", j+1, "/", nr_pruning_rounds)
        # train for nr_epochs_per_round steps
        # fit model and save accuracy in history
        history_trained_net_section = trained_model.fit(
            train_ds,
            validation_data = val_ds,
            epochs=nr_epochs_per_round
            #verbose = 0, 
        )
        
        
        # save history if its first round of pruning, add to existing history if not 
        if j == 0:
            history_trained_net = history_trained_net_section
        else:
            history_trained_net = add_section_to_his(history_trained_net, history_trained_net_section)
            
            
        # prune net according to pruning rates for this round
        trained_model = prune_conv2(trained_model, conv_pruning[j], dense_pruning[j], op_layer_pruning[j])
        
        
        # reinitialize pruned net
        for orig_wts, trained_wts in zip(orig_model.trainable_weights, trained_model.trainable_weights):
            trained_wts.assign(tf.where(tf.equal(trained_wts, 0), trained_wts, orig_wts))
            

    # final network is already a canditat for a WT as it was reinitialized, save it
    trained_model.save_weights("../tickets/WTs_SVHN/WT_s0.1_nr" + str(i) + ".h5")
    
    # save history_trained_net
    with open('../tickets/WTs_SVHN/his_train_0.1_nr' + str(i), 'wb') as file_pi:
        pickle.dump(history_trained_net.history, file_pi)
    
    # check if WT (save histories to make it possible to check and decide later on)
    compare_WT_to_orig(trained_model, orig_model, nr = i)
    

In [ ]:
# look at example history while IMP
plot_history(history_trained_net)